In [1]:
import numpy as np

In [16]:
def convolution3d(input_array, kernel_array, stride=1, padding=0, dilation=1, bias=True):
    if input_array.ndim != 3 or kernel_array.ndim != 3:
        raise ValueError("Invalid input: 3D arrays with 'NHW' layout expected.")

    if padding > 0:
        input_array = np.pad(input_array, ((padding, padding), (padding, padding), (padding, padding)), \
                             mode='constant', constant_values=0)

    input_depth, input_height, input_width = input_array.shape
    kernel_depth, kernel_height, kernel_width = kernel_array.shape

    output_depth = (input_depth - kernel_depth + 2 * padding) // stride + 1
    output_height = (input_height - kernel_height + 2 * padding) // stride + 1
    output_width = (input_width - kernel_width + 2 * padding) // stride + 1

    output_array = np.zeros((output_depth, output_height, output_width))

    for d in range(0, input_depth - kernel_depth * dilation + 1, stride):
        for i in range(0, input_height - kernel_height * dilation + 1, stride):
            for j in range(0, input_width - kernel_width * dilation + 1, stride):
                input_patch = input_array[d:d + kernel_depth * dilation, i:i + kernel_height * dilation, \
                                          j:j + kernel_width * dilation]
                output_array[d // stride, i // stride, j // stride] = np.sum(input_patch * kernel_array)

    if bias:
        bias_values = np.random.randn()
        output_array += bias_values

    return output_array

In [5]:
input_array = np.random.rand(5, 7, 7)
kernel_array = np.random.rand(3, 3, 3)

stride = 1
padding = 0

output_result1 = convolution3d(input_array, kernel_array, stride, padding)

Input Array:
[[[0.43522598 0.13602679 0.7230524  0.49791824 0.94755845 0.43076088
   0.33356364]
  [0.50527862 0.14360383 0.09826919 0.49633204 0.39764599 0.05693702
   0.35104435]
  [0.12275971 0.62742663 0.75850504 0.91042715 0.33581476 0.79511238
   0.42367157]
  [0.16615148 0.80397829 0.7806102  0.6920964  0.14075205 0.08394091
   0.08102631]
  [0.61745671 0.14012875 0.71368275 0.32846351 0.12297293 0.08141099
   0.89554586]
  [0.4299781  0.05252058 0.0754713  0.74043128 0.58645235 0.227782
   0.12432765]
  [0.98023233 0.23467415 0.40457872 0.72745627 0.0431159  0.42392814
   0.7367171 ]]

 [[0.817744   0.23832964 0.81506764 0.03098228 0.97498483 0.7459915
   0.23835261]
  [0.00676229 0.68982268 0.40014402 0.3492096  0.88559316 0.81990026
   0.11611186]
  [0.00734986 0.36874047 0.07716616 0.75702867 0.11460215 0.95487495
   0.01034953]
  [0.00336551 0.80168358 0.17115553 0.26472451 0.67112637 0.77109176
   0.13295535]
  [0.77484025 0.25803336 0.82673333 0.65257885 0.95249712 0.4735

In [6]:
import torch
import torch.nn.functional as F

def convolution3d(input_array, kernel_array, stride, padding):
    input_tensor = torch.tensor(input_array, dtype=torch.float32)
    kernel_tensor = torch.tensor(kernel_array, dtype=torch.float32)

    input_tensor = input_tensor.unsqueeze(0).unsqueeze(0) 
    kernel_tensor = kernel_tensor.unsqueeze(0).unsqueeze(0)  

    output_tensor = F.conv3d(input_tensor, kernel_tensor, stride=stride, padding=padding)
    output_result = output_tensor.squeeze().detach().numpy()
    return output_result

output_result2 = convolution3d(input_array, kernel_array, stride, padding)

In [17]:
print(output_result1 == output_result2)

True
